<a href="https://colab.research.google.com/github/pollinations/hive/blob/main/notebooks/1%20Text-To-Image/1%20LAION%20Latent%20Diffusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![A painting of a creature that is half octopus and half human](https://i.imgur.com/BxjhCmK.png)

Example: *A painting of a creature that is half octopus and half human*

Latent Diffusion model Text-to-image synthesis with 1.45B parameter model. This great model rivals OpenAI's DALL-E and can generate beautiful artistic styles as well as faces and text.

**Could work with the free mode of Google Colab now**

---
Credits: model released by [CompVis](https://github.com/CompVis/latent-diffusion) and trained on the [LAION-400M dataset](https://laion.ai/laion-400-open-dataset/)
Colab assembled by [@multimodalart](https://twitter.com/multimodalart)

In [ ]:
Prompt = "A painting of a squirrel eating a hotdog" #@param{type:"string"}

Steps = 100 #@param {type:"integer"}

# Apparently this parameter can be 0 or 1. Did not completely understand what it does yet.
ETA = 1 #@param{type:"integer"}

Iterations = 2 #@param{type:"integer"}

# How many images to generate in parallel. Two many could lead to memory issues
Samples_in_parallel=1 #@param{type:"integer"}

# As a rule of thumb, higher values of scale produce better samples at the cost of a reduced output diversity.
Diversity_scale=10.0 #@param {type:"number"}

PLMS_sampling=True #@param {type:"boolean"}

output_path = "/content"

In [ ]:
# Up to 512x512 should be possible without running out of memory

Width=256

# Up to 512x512 should be possible without running out of memory
Height=256

# Setup stuff

In [ ]:
#@title Installation
!sudo apt install -y aria2
!git clone https://github.com/crowsonkb/latent-diffusion.git
!git clone https://github.com/CompVis/taming-transformers
!pip install -e ./taming-transformers
!pip install omegaconf>=2.0.0 pytorch-lightning>=1.0.8 torch-fidelity einops
!pip install transformers
!pip install open_clip_torch
import sys
sys.path.append(".")
sys.path.append('./taming-transformers')
from taming.models import vqgan 

import os
os.makedirs(output_path, exist_ok=True)

Cloning into 'latent-diffusion'...
remote: Enumerating objects: 231, done.
remote: Counting objects: 100% (231/231), done.
remote: Compressing objects: 100% (155/155), done.
remote: Total 231 (delta 80), reused 185 (delta 50), pack-reused 0
Receiving objects: 100% (231/231), 24.10 MiB | 22.10 MiB/s, done.
Resolving deltas: 100% (80/80), done.
Cloning into 'taming-transformers'...
remote: Enumerating objects: 1335, done.
remote: Counting objects: 100% (525/525), done.
remote: Compressing objects: 100% (493/493), done.
remote: Total 1335 (delta 58), reused 481 (delta 30), pack-reused 810
Receiving objects: 100% (1335/1335), 412.35 MiB | 28.36 MiB/s, done.
Resolving deltas: 100% (268/268), done.
Obtaining file:///content/taming-transformers
  Running setup.py develop for taming-transformers
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires 

Now, download the checkpoint (~5.7 GB). This will usually take 3-6 minutes.

In [ ]:
#@title Download model
%cd latent-diffusion/ 


!mkdir -p /content/models/
if not os.path.exists("/content/models/ldm-model.ckpt"):
  !wget -O /content/models/ldm-model.ckpt https://public-ipfs-gateway.pollinations.ai/ipfs/QmUKTHwT9nWtPp1bes3VGjQRk8gizKvTo6s4cNgsoM1Jpk/model.ckpt

/content/latent-diffusion
--2022-04-05 12:25:56--  https://nftstorage.link/ipfs/QmUKTHwT9nWtPp1bes3VGjQRk8gizKvTo6s4cNgsoM1Jpk/model.ckpt
Resolving nftstorage.link (nftstorage.link)... 104.18.7.107, 104.18.6.107, 2606:4700::6812:66b, ...
Connecting to nftstorage.link (nftstorage.link)|104.18.7.107|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://bafybeicy26q4fimolgbw6gxrsfoh3p6hzgtab6sylb7a6frevyrbfpjtgu.ipfs.nftstorage.link/model.ckpt [following]
--2022-04-05 12:25:56--  https://bafybeicy26q4fimolgbw6gxrsfoh3p6hzgtab6sylb7a6frevyrbfpjtgu.ipfs.nftstorage.link/model.ckpt
Resolving bafybeicy26q4fimolgbw6gxrsfoh3p6hzgtab6sylb7a6frevyrbfpjtgu.ipfs.nftstorage.link (bafybeicy26q4fimolgbw6gxrsfoh3p6hzgtab6sylb7a6frevyrbfpjtgu.ipfs.nftstorage.link)... 104.18.6.107, 104.18.7.107, 2606:4700::6812:66b, ...
Connecting to bafybeicy26q4fimolgbw6gxrsfoh3p6hzgtab6sylb7a6frevyrbfpjtgu.ipfs.nftstorage.link (bafybeicy26q4fimolgbw6gxrsfoh3p6hzgtab6sylb7a6frevyrbfpjtgu

Let's also check what type of GPU we've got.

In [ ]:
!nvidia-smi

Load it.

In [ ]:
#@title loading utils
import torch
from omegaconf import OmegaConf

from ldm.util import instantiate_from_config

In [ ]:
#@title Import stuff
import argparse, os, sys, glob
import torch
import numpy as np
from omegaconf import OmegaConf
from PIL import Image
from tqdm.auto import tqdm, trange
tqdm_auto_model = __import__("tqdm.auto", fromlist=[None]) 
sys.modules['tqdm'] = tqdm_auto_model
from einops import rearrange
from torchvision.utils import make_grid
import transformers
import gc
from ldm.util import instantiate_from_config
from ldm.models.diffusion.ddim import DDIMSampler
from ldm.models.diffusion.plms import PLMSSampler
from open_clip import tokenizer
import open_clip


In [ ]:
#@title Load necessary functions
#NSFW CLIP Filter
clip_model, _, preprocess = open_clip.create_model_and_transforms('ViT-B-32-quickgelu', pretrained='laion400m_e32')
text = tokenizer.tokenize(["NSFW", "adult content", "porn", "naked people","genitalia","penis","vagina"])
with torch.no_grad():
  text_features = clip_model.encode_text(text)

#The higher this number the less the nsfw filter is sensitive
nsfw_scale = 18
def load_model_from_config(config, ckpt, verbose=False):
    print(f"Loading model from {ckpt}")
    pl_sd = torch.load(ckpt, map_location="cpu")
    sd = pl_sd["state_dict"]
    model = instantiate_from_config(config.model)
    m, u = model.load_state_dict(sd, strict=False)
    if len(m) > 0 and verbose:
        print("missing keys:")
        print(m)
    if len(u) > 0 and verbose:
        print("unexpected keys:")
        print(u)

    model = model.half().cuda()
    model.eval()
    return model

config = OmegaConf.load("configs/latent-diffusion/txt2img-1p4B-eval.yaml")  # TODO: Optionally download from same location as ckpt and chnage this logic
model = load_model_from_config(config, "/content/models/ldm-model.ckpt") 

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model = model.to(device)
def run(opt):
    torch.cuda.empty_cache()
    gc.collect()
    if opt.plms:
        opt.ddim_eta = 0
        sampler = PLMSSampler(model)
    else:
        sampler = DDIMSampler(model)
    
    os.makedirs(opt.outdir, exist_ok=True)
    outpath = opt.outdir

    prompt = opt.prompt


    sample_path = os.path.join(outpath, "samples")
    os.makedirs(sample_path, exist_ok=True)
    base_count = len(os.listdir(sample_path))

    all_samples=list()
    with torch.no_grad():
        with torch.cuda.amp.autocast():
            with model.ema_scope():
                uc = None
                if opt.scale > 0:
                    uc = model.get_learned_conditioning(opt.n_samples * [""])
                for n in trange(opt.n_iter, desc="Sampling"):
                    c = model.get_learned_conditioning(opt.n_samples * [prompt])
                    shape = [4, opt.H//8, opt.W//8]
                    samples_ddim, _ = sampler.sample(S=opt.ddim_steps,
                                                    conditioning=c,
                                                    batch_size=opt.n_samples,
                                                    shape=shape,
                                                    verbose=False,
                                                    unconditional_guidance_scale=opt.scale,
                                                    unconditional_conditioning=uc,
                                                    eta=opt.ddim_eta)

                    x_samples_ddim = model.decode_first_stage(samples_ddim)
                    x_samples_ddim = torch.clamp((x_samples_ddim+1.0)/2.0, min=0.0, max=1.0)

                    for x_sample in x_samples_ddim:
                        x_sample = 255. * rearrange(x_sample.cpu().numpy(), 'c h w -> h w c')
                        Image.fromarray(x_sample.astype(np.uint8)).save(os.path.join(sample_path, f"{base_count:04}.png"))
                        base_count += 1
                    all_samples.append(x_samples_ddim)


    # additionally, save as grid
    grid = torch.stack(all_samples, 0)
    grid = rearrange(grid, 'n b c h w -> (n b) c h w')
    grid = make_grid(grid, nrow=opt.n_samples)

    # to image
    grid = 255. * rearrange(grid, 'c h w -> h w c').cpu().numpy()
    
    Image.fromarray(grid.astype(np.uint8)).save(os.path.join(outpath, f'{prompt.replace(" ", "-")}.png'))
    display(Image.fromarray(grid.astype(np.uint8)))
    #print(f"Your samples are ready and waiting four you here: \n{outpath} \nEnjoy.")

# Do the run

In [ ]:
#@title Parameters
import argparse


args = argparse.Namespace(
    prompt = Prompt, 
    outdir=output_path,
    ddim_steps = Steps,
    ddim_eta = ETA,
    n_iter = Iterations,
    W=Width,
    H=Height,
    n_samples=Samples_in_parallel,
    scale=Diversity_scale,
    plms=PLMS_sampling
)
run(args)